In [1]:
import cv2
import numpy as np

In [2]:
from keras.datasets import cifar100

2024-12-08 17:27:25.766464: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Define the number of bins for the histogram
num_bins = 256

In [131]:
# Define HOG parameters
cell_size = (4, 4)           # Size of each cell in pixels
block_size = (2, 2)          # Number of cells in a block
block_stride = (4, 4)        # Step size for block movement
win_size = (32, 32)          # Same as image size
nbins = 9                    # Number of orientation bins

# Initialize HOG descriptor
hog = cv2.HOGDescriptor(
    _winSize=(win_size[1], win_size[0]),
    _blockSize=(block_size[1] * cell_size[1], block_size[0] * cell_size[0]),
    _blockStride=(block_stride[1], block_stride[0]),
    _cellSize=(cell_size[1], cell_size[0]),
    _nbins=nbins
)

# Function to compute HOG features for the dataset
def compute_hog_features(images):
    features = []
    for img in images:
        hog_feats = hog.compute(img).flatten()  # Flatten to 1D vector
        features.append(hog_feats)
    return np.array(features)



In [132]:
# Convert images to grayscale
x_train_gray = np.array([cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in x_train])
x_test_gray = np.array([cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in x_test])

# Extract HOG features for training and test datasets
x_train_hog = compute_hog_features(x_train_gray)
x_test_hog = compute_hog_features(x_test_gray)

print(f"Training HOG feature shape: {x_train_hog.shape}")
print(f"Test HOG feature shape: {x_test_hog.shape}")

Training HOG feature shape: (50000, 1764)
Test HOG feature shape: (10000, 1764)


In [133]:
y_train.shape

(50000, 1)

In [134]:
from sklearn.decomposition import PCA
pca1 = PCA(n_components=1)
pca2 = PCA(n_components=1)
# Fit the PCA model to the data
pca1.fit(x_train_hog)

# Transform the data to the new lower-dimensional space
x_train_hog = pca1.transform(x_train_hog)

pca2.fit(x_test_hog)

# Transform the data to the new lower-dimensional space
x_test_hog = pca2.transform(x_test_hog)

print(f"Training HOG feature shape: {x_train_hog.shape}")
print(f"Test HOG feature shape: {x_test_hog.shape}")

Training HOG feature shape: (50000, 1)
Test HOG feature shape: (10000, 1)


In [129]:
# Function to extract color histogram features
def color_histogram(image):
    # Convert image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

    # Calculate histograms for each color channel
    hist_h = cv2.calcHist([hsv_image], [0], None, [num_bins], [0, 180])  # Hue
    hist_s = cv2.calcHist([hsv_image], [1], None, [num_bins], [0, 256])  # Saturation
    hist_v = cv2.calcHist([hsv_image], [2], None, [num_bins], [0, 256])  # Value

    # Normalize histograms
    cv2.normalize(hist_h, hist_h, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    cv2.normalize(hist_s, hist_s, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    cv2.normalize(hist_v, hist_v, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)

    # Concatenate histograms into a single feature vector
    histogram = np.concatenate((hist_h, hist_s, hist_v), axis=0)

    return histogram

In [130]:
# Extract features from training and test data
train_features_color = []
for image in x_train:
    feature_c = color_histogram(image)
    train_features_color.append(feature_c)

test_features_color = []
for image in x_test:
    feature_c_test = color_histogram(image)
    test_features_color.append(feature_c_test)

# Convert feature lists to NumPy arrays
train_features_color = np.array(train_features_color)
test_features_color = np.array(test_features_color)

print("Training features shape:", train_features_color.shape)
print("Test features shape:", test_features_color.shape)

Training features shape: (50000, 768, 1)
Test features shape: (10000, 768, 1)


In [121]:
# Function to compute LBP histogram
def lbp_histogram(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply LBP operator
    lbp = cv2.calcHist([gray], [0], None, [256], [0, 256])
    lbp = cv2.normalize(lbp, lbp, 0, 255, cv2.NORM_MINMAX)

    # Flatten the histogram
    hist = lbp.flatten()

    return hist

In [125]:
# Extract features from training
train_features_lbp = []
for image in x_train:
    feature_lbp = lbp_histogram(image)
    train_features_lbp.append(feature_lbp)
    
# Extract features from test data
test_features_lbp = []
for image in x_test:
    feature_lbp = lbp_histogram(image)
    test_features_lbp.append(feature_lbp)
    
# Convert feature lists to NumPy arrays
train_features_lbp = np.array(train_features_lbp)
test_features_lbp = np.array(test_features_lbp)

print("Training features shape:", train_features_lbp.shape)
print("Test features shape:", test_features_lbp.shape)

Training features shape: (50000, 256)
Test features shape: (10000, 256)


In [127]:
pca3 = PCA(n_components=1)
pca4 = PCA(n_components=1)
# Fit the PCA model to the data
pca3.fit(train_features_lbp)

# Transform the data to the new lower-dimensional space
train_features_lbp = pca3.transform(train_features_lbp)

pca4.fit(test_features_lbp)

# Transform the data to the new lower-dimensional space
test_features_lbp = pca4.transform(test_features_lbp)

print(f"Training HOG feature shape: {train_features_lbp.shape}")
print(f"Test HOG feature shape: {test_features_lbp.shape}")

Training HOG feature shape: (50000, 1)
Test HOG feature shape: (10000, 1)


In [41]:
import matplotlib.pyplot as plt
# Select a random image from the training set
#index = np.random.randint(len(x_train))
#img = x_train[index]

#feature_lbp=feature_lbp.reshape(32, 32)
#feature_lbp = lbp_histogram(img)

#plt.imshow(img)
#plt.imshow(feature_lbp)

plt.figure()
plt.scatter 

<function matplotlib.pyplot.scatter(x, y, s=None, c=None, marker=None, cmap=None, norm=None, vmin=None, vmax=None, alpha=None, linewidths=None, *, edgecolors=None, plotnonfinite=False, data=None, **kwargs)>

<Figure size 640x480 with 0 Axes>

In [1]:
from math import sqrt
class KNN():
  def __init__(self,k):
    self.k=k
    
  def fit(self,X_train,y_train):
    self.x_train=X_train
    self.y_train=y_train
    
  def calculate_euclidean(self,sample1,sample2):
    distance=0.0
    for i in range(len(sample1)):
      distance+=(sample1[i]-sample2[i])**2 #Euclidean Distance = sqrt(sum i to N (x1_i – x2_i)^2)
    return sqrt(distance)

  def nearest_neighbors(self,test_sample):
    distances=[]#calculate distances from a test sample to every sample in a training set
    for i in range(len(self.x_train)):
      distances.append((self.y_train[i],self.calculate_euclidean(self.x_train[i],test_sample)))
    distances.sort(key=lambda x:x[1])#sort in ascending order, based on a distance value
    neighbors=[]
    for i in range(self.k): #get first k samples
      neighbors.append(distances[i][0])
    return neighbors

  def predict(self,test_set):
    predictions=[]
    for test_sample in test_set:
      neighbors=self.nearest_neighbors(test_sample)
      labels=[sample for sample in neighbors]
      prediction=max(labels,key=labels.count)
      predictions.append(prediction)
    return predictions

In [145]:
clf=KNN(10)
clf.fit(x_train_hog,y_train)
predictions=clf.predict(x_test_hog)


KeyboardInterrupt: 

In [114]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, predictions) #our model
print(cm)

print (accuracy_score(y_test, predictions))

[[0 3 0 ... 0 1 0]
 [1 1 0 ... 0 0 0]
 [3 2 2 ... 2 2 0]
 ...
 [2 1 1 ... 3 0 0]
 [1 0 2 ... 1 2 0]
 [1 0 0 ... 1 1 2]]
0.0204


In [117]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 10, metric = 'minkowski', p = 2)#The default metric is minkowski, and with p=2 is equivalent to the standard Euclidean metric.
classifier.fit(x_train_hog, y_train.ravel())

y_pred = classifier.predict(x_test_hog)
accuracy_score(y_test, y_pred)

0.0203

In [147]:
clf2=KNN(10)
clf2.fit(train_features_lbp,y_train)
predictions_2=clf2.predict(test_features_lbp)
print (accuracy_score(y_test, predictions_2))

0.0159


In [148]:
classifier = KNeighborsClassifier(n_neighbors = 10, metric = 'minkowski', p = 2)#The default metric is minkowski, and with p=2 is equivalent to the standard Euclidean metric.
classifier.fit(train_features_lbp, y_train.ravel())

y_pred = classifier.predict(test_features_lbp)
accuracy_score(y_test, y_pred)

0.0159